This file contains the code of the altered spell check program.

The first section of code was left unaltered. In previous alterations of my code added other training sets and other edit functions. My intent was to expand the dictionatry NWORDS as well as refine the way candidates are selected for the corrector fucntion. However, most of my other implementations did far wrose than the current implementation.

In [20]:
import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(file('big.txt').read()))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def addWord(word):
        NWORDS[word] =+ 1

The biggest change I made to the program was to make changes to the correct function. The biggest key that I focused on was to have a "training set that cannot be saved in memeory." Now I originally intedned to focus on using web cralwers to randomly find some article and use that as a trining set and then fetching another article and using that as a secondary training set, but unfortunately I was not skilled in Python enough to do that. So instead I opted to have an interactive program in which you can add words into the dictionary. I also made it so that the user can interactively iterate through other likely candidates for corrections. If the user finds the word they intended to spell, then the count value is incremented to make it a more likely candidate for next time. So long as the program is running, the program is learning based on the user input.

This user input model is a bit impractical due to the fact that it always has to be running in order to correctly learn. Improvment to this would be to save learn states and to have the program remember what it learned previously. This program is a slight improvement over the orginal in that it at least allows users to alter the probablilties of encountering words, even if it's ever so slightly.

In [21]:
def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    retVal = max(candidates, key=NWORDS.get)   
    isValue = input("Did you mean? " + retVal +"\n1.Yes\n2.No\n")
    if isValue == 1:
        return retVal
    else:
        willAdd = input("Do you wish to add this as a new word?\n1.yes\n2.No\n")
        if willAdd == 1:
            addWord(word)
    while isValue == 2:
        if retVal in candidates: candidates.remove(retVal)
        retVal = max(candidates, key=NWORDS.get)
        isValue = input("Did you mean? " + retVal +"\n1.Yes\n2.No\n")
        if isValue == 1:
            addWord(retVal)
            return retVal

Given more time I would improve this program by producing a prefix bank and add it to an edit function or make a new function entirely. What I mean by that is that I would list off endings such as -ed, -ing, -ly, and such and use that to make a list of likely candidates. I feel like that would ha

In [22]:

def main():
    done = 2
    while done == 2:
        print("Welcome to spell checker!\nPlease type in a word and I'll check its spelling!")
        myWord = raw_input()
        correct(myWord)
        done = input("Are You finished?\n1.Yes\n2.no\n")
        
if __name__ == "__main__":
    main()

Welcome to spell checker!
Please type in a word and I'll check its spelling!
invented
Did you mean? invented
1.Yes
2.No
1
Are You finished?
1.Yes
2.no2
Welcome to spell checker!
Please type in a word and I'll check its spelling!
invested
Did you mean? invested
1.Yes
2.No
1
Are You finished?
1.Yes
2.no2
Welcome to spell checker!
Please type in a word and I'll check its spelling!
inbented
Did you mean? invented
1.Yes
2.No
1
Are You finished?
1.Yes
2.no1
